In [5]:
import percolate as p8
from IPython.display import Markdown
from percolate.models.p8 import PercolateAgent, Task, ResearchIteration, Function

from percolate.utils.studio import add_percolate_endpoints

# """add some endpoints as functions to percolate api"""
# add_percolate_endpoints([#'create_task_tasks__post', 
#                          'execute_research_iteration_tasks_research_execute_post'])

#agent = p8.Agent(ResearchIteration,allow_help=True)
agent = p8.Agent(Task)

#agent("What comments are associate with T1234")
# p8.repository(Function).update_records(
#     [
#         Function.from_entity(Task),
#         Function.from_entity(ResearchIteration),
#     ]
# )
# language_model='claude-3-7-sonnet-20250219'
#Markdown(Task.get_model_description(True))

import percolate as p8
from percolate.models.p8 import Task, ResearchIteration
agent = p8.Agent(Task)
agent("please execute a research task to compare google to the east india company")

In [ ]:
agent.messages.data

In [2]:
d = {'tags': ['Tasks'], 'summary': 'Execute Research Iteration', 'description': 'execute a research plan - perform each search in the question set - this can take time so should be done as background tasks', 'operationId': 'execute_research_iteration_tasks_research_execute_post', 'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/ResearchIteration'}}}, 'required': True}, 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {'$ref': '#/components/schemas/ResearchIteration'}}}}, '422': {'description': 'Validation Error', 'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'}}}}}, 'parameters': [], 'request_body': {'properties': {'id': {'anyOf': [{'type': 'string', 'format': 'uuid'}, {'type': 'string'}, {'type': 'null'}], 'title': 'Id', 'default': 'unique id for rel'}, 'iteration': {'type': 'integer', 'title': 'Iteration'}, 'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Content', 'description': 'An optional summary of the results discovered', 'embedding_provider': 'default'}, 'conceptual_diagram': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Conceptual Diagram', 'description': 'The mermaid diagram for the plan - typically generated in advanced of doing a search'}, 'question_set': {'items': {'$ref': '#/components/schemas/_QuestionSet'}, 'type': 'array', 'title': 'Question Set', 'description': 'a set of questions and their ids from the conceptual diagram'}, 'task_id': {'anyOf': [{'type': 'string', 'format': 'uuid'}, {'type': 'string'}, {'type': 'null'}], 'title': 'Task Id', 'default': 'Research are linked to tasks which are at minimum a question'}}, 'type': 'object', 'required': ['iteration', 'question_set'], 'title': 'ResearchIteration', 'description': 'A research iteration is a plan to deal with a task.\nIf you are asked for a plan you should first use your json structure to create a plan. \nSearches should be done via the webs search. \n1. If you are asked to search for existing plans you can use the _search_ method to find plans themselves\n2. if you are asked to search for general topics, you should execute the web search\nYou can generate conceptual diagrams using mermaid diagrams to provide an overview of the research plan.\nWhen you generate a conceptual plan you should link question sets to plans for example each question should have labels that link to part of the conceptual diagram using the mermaid diagram format to describe your plan.'}}
d

In [3]:
from percolate.services.OpenApiService import OpenApiSpec
uri =  f"http://percolate-api:5008/openapi.json"
service = OpenApiSpec(uri)
service

In [42]:
import re
import json
R
def parse_fenced_code_blocks(
    input_string, try_parse=True, select_type="json", first=True, on_error=None
):
    """
    extract code from fenced blocks - will try to parse into python dicts if option set
    json is assumed
    """
    try:
        input_string = input_string.replace("\n", "")
        pattern = r"```(.*?)```|~~~(.*?)~~~"
        matches = re.finditer(pattern, input_string, re.DOTALL)
        code_blocks = []
        for match in matches:
            code_block = match.group(1) if match.group(1) else match.group(2)
            # print(code_block)
            if code_block[: len(select_type)] == select_type:
                code_block = code_block[len(select_type) :]
                code_block.strip()
                if try_parse and select_type == "json":
                    code_block = json.loads(code_block)
                code_blocks.append(code_block)
        return code_blocks if not first and len(code_blocks) > 1 else code_blocks[0]
    except:
        if on_error:
            raise
        # raise
        # FAIL SILENT
        return [] if not first else {}
parse_fenced_code_blocks(R)

In [46]:
from percolate.services import PostgresService
import html2text
import json
ps = PostgresService()
data = ps.execute(""" select id,content from p8."Resources" 
order by created_at desc
limit 25""")
from percolate.models.p8 import Task, ResearchIteration
agent = p8.Agent(Task,allow_help=False)
sets = []
for d in data:
    try:
        #print(d['content'])
        R = agent.run(f""" please explain the data below according to the task of comparing Google to the east india company and return a json object with two properties. 
        1. extract entity `tags` in a path format from specific to general like Person/CEOs or LLMs/AI and add them as a list of tags (5-7). the entities should be relevant to the tasl
        2. return a nested dictionary called `analysis` with 3 fields; `discussion`, `category`, `questions`. for example if the data provide a history of east india business practices you can label it as such, list the sorts of questions addressed in the section 
         and then provide a comprehensive (three comprehensive and detailed paragraph e.g. A4 page of content) discussion of either google, the easy india company or their relationship

        # Data
        ```{d['content']}``` 
        """)
        sets.append(parse_fenced_code_blocks(R))
    except Exception as ex:
        print(f""" ERROR  {ex} """)

sets
# for _id in ids:
#     try:
# #         id = _id['id']
# #         data = ps.execute(f"""select id,content from p8."Resources" where id = '{id}'""")[0]['content']
# #         data = html2text.html2text(data)
# #         ps.execute(f""" update p8."Resources" set content = %s where id = %s""", data=(data,id))
# #         print(id)
 
#     except:
#         print(id)
#         raise
# data

In [68]:
data = agent.run(f""" Using just the provided data and withuut using tools, generate a detailed SCATHING reports with clear sections in markdown on the topic of compariing google to the east india company.
Make clear the questions we are setting out to address, have sections that break down key discussions points, an intro and conclusion. Be as detailed as possible.
Your report should not be bullet points but flesh out a comprehensive discussion. It should be several A4 pages in length. In this case I want you to 
be critical of both companies bsiness practices and be cynical and humorous in your writing style. This is not intended to be a very serious report just a social commentary
```{sets}``` """,
language_model='claude-3-7-sonnet-20250219'
                )
data

In [89]:
ls = [s.get('tags',[]) for s in sets]
lss = []
for i in ls:
    for j in i:
        lss.append(f"T4/{j}")
jd = json.dumps(lss)
ps.execute(f"""  SELECT p8.create_graph_from_paths('{jd}'::jsonb); """)

In [69]:
with open("/Users/sirsh/Downloads/report_cyn.md", 'w') as f:
    f.write(data)

In [70]:
Markdown(data)

In [58]:
data = agent.run(f""" please generate a mermaid diagram summarsing the main arguments in this data - can you maintain google and east india company as two entitites and show the relationship in shared nodes. Dont add explanation. Google and EIC should be top level with shared nodes and color coding should isolate them. please add detailed node breakdown showing key comparisons with details```{sets}``` """)
data

In [59]:
d = """A[Google]:::google\n    B[East India Company (EIC)]:::eic\n\n    A -->|Global Influence| C[Monopolistic Practices]\n    B -->|Global Influence| C\n\n    A -->|Economic Impact| D[Impact on Local Economies]\n    B -->|Economic Impact| D\n\n    A -->|Technological Innovation| E[Innovation and Expansion]\n    B -->|Military and Political Power| E\n\n    A -->|Data and Privacy Concerns| F[Ethical Considerations]\n    B -->|Exploitation and Colonialism| F\n\n    A -->|Digital Transformation| G[Societal Impact]\n    B -->|Cultural and Social Impact| G\n\n    A -->|Advertising and Revenue| H[Business Model]\n    B -->|Trade and Revenue| H\n\n    A -->|Regulatory Scrutiny| I[Challenges and Criticisms]\n    B -->|Government Intervention| I\n\n    classDef google fill:#f9f,stroke:#333,stroke-width:2px;\n    classDef eic fill:#bbf,stroke:#333,stroke-width:2px;\n"""
with open ("/Users/sirsh/Downloads/mm.txt", 'w') as f:
    f.write(d)

In [50]:
with open("/Users/sirsh/Downloads/geast_report.json", 'w') as f:
    json.dump(sets,f)

In [2]:
task_agent = p8.Agent(Task)
#task_agent.run("Please go ahead and create a research iteration to research tech news. do not tell me how to do it")
task_agent.run("please generate a research plan to research latest tech topics. Do not tell me how to do it, just do it")
#Markdown(task_agent.run("Please go ahead and generate a full research iteration plan to get latest tech news in json format - you do not need to execute, just show me the structure"))

In [ ]:
#task_agent.run("please perform a search for tech news and write a report")

In [ ]:
# from percolate.utils.studio import add_percolate_endpoints
# add_percolate_endpoints(['create_task_tasks__post', 
#                              'execute_research_iteration_tasks_research_execute_post'])

In [10]:
p8.get_proxy(f.proxy_uri)

In [8]:
task_agent._function_manager.plan("A function to do research iterations")

In [5]:
"""TODO check if agent knows its json structure

- i think in the database we generate the prompt but we need to do this in code too so it knows. test.
- then we need to test that crud works i.e. that the save object can be used to save self objects
- then we need to test that we can post to ebdpoints that use this type
"""
from IPython.display import Markdown
#Markdown(agent("Please create a research plan to search for tech news using your structured object format as Json"))
agent('find a function to perform research iterations and use it to get the latest tech news')

In [11]:
# from IPython.display import Markdown
d= agent.run("please search the 'matter with things' and provide a detailed report is doscoursive discussion format about what you find by visiting a number of the recovered search result details without announcing text generation. you can use your own knolwedge to reflect and extend what you find and related to other things - generate your report in one pass",
            language_model='claude-3-7-sonnet-20250219')
             
Markdown(d)

In [ ]:
# create a better database function that takes a task id and then allow the DB research iteration to run or be recruited by a task

In [ ]:
# practice saving structured -> crud in the database and in python

In [ ]:
# pratice recruitement - starting with a task - in the database and in python|

In [ ]:
# practice tasks queues with database triggers

In [ ]:
# test resume session in the agent when we finished prem